AC with CartPole V0

In [1]:
import os

import random
import gym
import pylab
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

#tf.config.experimental_run_functions_eagerly(True)
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import copy

EPISODES = 100000 # Number of times the enviroment is ran
LR = 0.02 # Learning rate 
NUM_ACTIONS = 2 # Number of possible action in the environment
NUM_STATES = 4 # Number of possible states in the exvironment
EPOCHS = 10 # Epochs to train the network (recommended between 3 and 30)
BATCH_SIZE = 64 # Batch size for the neural nets
BUFFER_SIZE = 2048 # Buffer of experiences
SHUFFLE = True # Whether to shuffle data or not while training
OPTIMIZER = Adam # Optimizer for both actor and critic
GAMMA = 0.99 # Used for the estimated reward
LAMBDA = 0.95 # Used in the original paper un the GAE
NORMALIZE = True # Whether to normalize GAE or not



# Create the actor used to select the action given an state
class Actor_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Softmax as there are different probabilities depending on the action
        output = Dense(NUM_ACTIONS, activation="softmax")(X)
        
        # Compile the model with the custom loss
        self.model = Model(inputs = X_input, outputs = output)
        self.model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER(lr=LR))

# Create the critic which will criticise how the actor is performing    
class Critic_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Linear output to know how good the action is
        value = Dense(1)(X)
        
        # Compile it with mse loss and gradient descent
        self.model = Model(inputs=X_input, outputs = value)
        self.model.compile(loss='mse', optimizer=OPTIMIZER(lr=LR))

# Combine both Actor and Critic to create the agent
class PPOAgent:
    def __init__(self, env_name):
        # Environment parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.episode = 0 # used to track current number episoded since start
        self.max_average = 0 # record max average reached
        
        # Used to plot a grapgh of the train process
        self.scores_, self.average_ = [], []

        # Create Actor-Critic network models
        self.Actor = Actor_Model()
        self.Critic = Critic_Model()
    
        # Names for the models
        self.Actor_name = f"{self.env_name}_PPO_Actor.h5"
        self.Critic_name = f"{self.env_name}_PPO_Critic.h5"

    # Get the action given the current state    
    def act(self, state):
        # Use the network to predict the next action to take, using the model
        prediction = self.Actor.model.predict(state)[0]
        
        # Probability based to choose the action
        action = np.random.choice(NUM_ACTIONS, p=prediction)
        action_onehot = np.zeros([NUM_ACTIONS])
        action_onehot[action] = 1
        return action, action_onehot, prediction

    # Generalized Advantage Estimation implemented in the original paper
    def get_gaes(self, rewards, dones, values, next_values):
        # Dones are used to track when is the final step of an episode, so next values are no applied
        deltas = [r + GAMMA * (1 - d) * nv - v for r, d, nv, v in zip(rewards, dones, next_values, values)]
        
        # Convert list to array as .mean() and .std() are used later
        deltas = np.stack(deltas)
        gaes = copy.deepcopy(deltas)
        
        for t in reversed(range(len(deltas) - 1)):
            gaes[t] = gaes[t] + (1 - dones[t]) * GAMMA * LAMBDA * gaes[t + 1]

        target = gaes + values
        if NORMALIZE:
            gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8)
        return np.vstack(gaes), np.vstack(target)

    def replay(self, states, actions, rewards, predictions, dones, next_states):
        # Reshape memory to appropriate shape for training
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        predictions = np.vstack(predictions)

        # Get Critic network predictions for state and next state
        values = self.Critic.model.predict(states)
        next_values = self.Critic.model.predict(next_states)

        # Get the advantage
        advantages, target = self.get_gaes(rewards, dones, np.squeeze(values), np.squeeze(next_values))

        # Stack info to unpack it in the custom loss
        y_true = actions 
        advantages = np.reshape(advantages, (2048))

        start_train = time.time()
        # Training Actor and Critic networks
        a_loss = self.Actor.model.fit(states, y_true, sample_weight=advantages, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        c_loss = self.Critic.model.fit(states, target, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        #print('Time: ', time.time()-start_train)
 
    def load(self):
        self.Actor.Actor.load_weights(self.Actor_name)
        self.Critic.Critic.load_weights(self.Critic_name)

    def save(self):
        self.Actor.model.save_weights(self.Actor_name)
        self.Critic.model.save_weights(self.Critic_name)

    def run_batch(self): # train every self.Training_batch episodes
        global LR
        state = self.env.reset()
        state = np.reshape(state, [1, NUM_STATES])
        done, score = False, 0
        finished = False
        while finished == False:
            # Instantiate or reset games memory
            states, next_states, actions, rewards, predictions, dones = [], [], [], [], [], []
            for t in range(BUFFER_SIZE):
                #self.env.render()
                # Actor picks an action
                action, action_onehot, prediction = self.act(state)
                # Retrieve new state, reward, and whether the state is terminal
                next_state, reward, done, _ = self.env.step(action)
                # Memorize (state, action, reward) for training
                states.append(state)
                next_states.append(np.reshape(next_state, [1, NUM_STATES]))
                actions.append(action_onehot)
                rewards.append(reward)
                dones.append(done)
                predictions.append(prediction)
                # Update current state
                state = np.reshape(next_state, [1, NUM_STATES])
                score += reward
                if done:
                    self.episode += 1
                    self.scores_.append(score)
                    #average, SAVING = self.PlotModel(score, self.episode)
                    #print("episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, EPISODES, score, average, SAVING))
                    if self.episode >= 100:
                        average = sum(self.scores_[-100:])/100
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, average))
                        if average > self.max_average:
                            self.max_average = average
                            if self.max_average > 150:
                                self.save()
                            #LR *= 0.99
                            #K.set_value(self.Actor.model.optimizer.learning_rate, LR)
                            #K.set_value(self.Critic.model.optimizer.learning_rate, LR)
                            
                        if average > 200:
                            plt.plot(self.scores_)
                            plt.xlabel("Episode")
                            plt.ylabel("Score")
                            finished = True
                            break

                    else:
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, sum(self.scores_)/len(self.scores_)))
                    
                    
                    state, done, score = self.env.reset(), False, 0
                    state = np.reshape(state, [1, NUM_STATES])
                    
            self.replay(states, actions, rewards, predictions, dones, next_states)
            if self.episode >= EPISODES:
                break
        self.env.close()  
                    
if __name__ == "__main__":
    start = time.time()
    env_name = 'CartPole-v0'
    agent = PPOAgent(env_name)
    #agent.run() # train as PPO, train every epesode
    agent.run_batch() # train as PPO, train every batch, trains better
    #agent.run_multiprocesses(num_worker = 8)  # train PPO multiprocessed (fastest)
    #agent.test()
    print((time.time() - start)/60)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Episode:     1		score:   14.00		average:   14.00
Episode:     2		score:   30.00		average:   22.00
Episode:     3		score:   14.00		average:   19.33
Episode:     4		score:   13.00		average:   17.75
Episode:     5		score:   67.00		average:   27.60
Episode:     6		score:   33.00		average:   28.50
Episode:     7		score:   25.00		average:   28.00
Episode:     8		score:   41.00		average:   29.62
Episode:     9		score:   29.00		average:   29.56
Episode:    10		score:   31.00		average:   29.70
Episode:    11		score:   12.00		average:   28.09
Episode:    12		score:   15.00		average:   27.00
Episode:    13		score:   16.00		average:   26.15
Episode:    14		score:   41.00		average:   27.21
Episode:    15		score:   27.00		average:   27.20
Episode:    16		score:   17.00		average:   26.56
Episode:    17		score:   14.00		average:   25.82
Episode:    18		score:   18.00		average:   25.39
Episode:    19		score:   54.00		avera

Episode:   170		score:   10.00		average:   11.61
Episode:   171		score:   11.00		average:   11.59
Episode:   172		score:    9.00		average:   11.47
Episode:   173		score:   12.00		average:   11.37
Episode:   174		score:   10.00		average:   11.31
Episode:   175		score:   10.00		average:   11.09
Episode:   176		score:   10.00		average:   10.96
Episode:   177		score:    9.00		average:   10.94
Episode:   178		score:    9.00		average:   10.72
Episode:   179		score:    9.00		average:   10.51
Episode:   180		score:    9.00		average:   10.35
Episode:   181		score:   10.00		average:   10.33
Episode:   182		score:    8.00		average:   10.25
Episode:   183		score:   11.00		average:   10.22
Episode:   184		score:   10.00		average:   10.23
Episode:   185		score:   10.00		average:   10.24
Episode:   186		score:   10.00		average:   10.24
Episode:   187		score:   11.00		average:   10.12
Episode:   188		score:   10.00		average:   10.13
Episode:   189		score:    8.00		average:   10.13
Episode:   190		scor

Episode:   338		score:   50.00		average:   31.00
Episode:   339		score:   51.00		average:   31.42
Episode:   340		score:   39.00		average:   31.71
Episode:   341		score:   55.00		average:   32.16
Episode:   342		score:  186.00		average:   33.94
Episode:   343		score:   67.00		average:   34.52
Episode:   344		score:  105.00		average:   35.47
Episode:   345		score:  198.00		average:   37.36
Episode:   346		score:   37.00		average:   37.65
Episode:   347		score:   50.00		average:   38.06
Episode:   348		score:   41.00		average:   38.37
Episode:   349		score:   52.00		average:   38.78
Episode:   350		score:   53.00		average:   39.22
Episode:   351		score:   45.00		average:   39.57
Episode:   352		score:   44.00		average:   39.82
Episode:   353		score:   63.00		average:   40.36
Episode:   354		score:   51.00		average:   40.77
Episode:   355		score:   44.00		average:   41.12
Episode:   356		score:   42.00		average:   41.45
Episode:   357		score:   35.00		average:   41.72
Episode:   358		scor

Episode:   506		score:    9.00		average:    9.46
Episode:   507		score:   10.00		average:    9.46
Episode:   508		score:    9.00		average:    9.45
Episode:   509		score:    9.00		average:    9.44
Episode:   510		score:    8.00		average:    9.43
Episode:   511		score:   10.00		average:    9.42
Episode:   512		score:    9.00		average:    9.41
Episode:   513		score:    9.00		average:    9.40
Episode:   514		score:   10.00		average:    9.41
Episode:   515		score:   10.00		average:    9.42
Episode:   516		score:    9.00		average:    9.42
Episode:   517		score:   10.00		average:    9.44
Episode:   518		score:    9.00		average:    9.45
Episode:   519		score:    8.00		average:    9.43
Episode:   520		score:   10.00		average:    9.44
Episode:   521		score:    8.00		average:    9.44
Episode:   522		score:    9.00		average:    9.44
Episode:   523		score:    8.00		average:    9.41
Episode:   524		score:    9.00		average:    9.42
Episode:   525		score:    9.00		average:    9.41
Episode:   526		scor

Episode:   683		score:    8.00		average:    9.32
Episode:   684		score:    9.00		average:    9.31
Episode:   685		score:    9.00		average:    9.30
Episode:   686		score:    9.00		average:    9.30
Episode:   687		score:    9.00		average:    9.29
Episode:   688		score:   10.00		average:    9.28
Episode:   689		score:   10.00		average:    9.29
Episode:   690		score:    9.00		average:    9.28
Episode:   691		score:    9.00		average:    9.28
Episode:   692		score:    9.00		average:    9.29
Episode:   693		score:    9.00		average:    9.30
Episode:   694		score:   10.00		average:    9.30
Episode:   695		score:    9.00		average:    9.31
Episode:   696		score:    8.00		average:    9.28
Episode:   697		score:    8.00		average:    9.26
Episode:   698		score:    9.00		average:    9.25
Episode:   699		score:    9.00		average:    9.25
Episode:   700		score:    9.00		average:    9.25
Episode:   701		score:    9.00		average:    9.25
Episode:   702		score:   10.00		average:    9.25
Episode:   703		scor

...

Episode:  7837		score:   10.00		average:    9.41
Episode:  7838		score:   10.00		average:    9.41
Episode:  7839		score:   10.00		average:    9.41
Episode:  7840		score:   10.00		average:    9.42
Episode:  7841		score:   10.00		average:    9.42
Episode:  7842		score:    9.00		average:    9.41
Episode:  7843		score:    8.00		average:    9.40
Episode:  7844		score:   10.00		average:    9.41
Episode:  7845		score:    9.00		average:    9.42
Episode:  7846		score:   10.00		average:    9.43
Episode:  7847		score:    9.00		average:    9.42
Episode:  7848		score:   10.00		average:    9.43
Episode:  7849		score:    9.00		average:    9.42
Episode:  7850		score:    9.00		average:    9.41
Episode:  7851		score:   10.00		average:    9.41
Episode:  7852		score:    9.00		average:    9.40
Episode:  7853		score:    9.00		average:    9.40
Episode:  7854		score:   10.00		average:    9.40
Episode:  7855		score:   10.00		average:    9.41
Episode:  7856		score:   10.00		average:    9.42
Episode:  7857		scor

Episode:  8011		score:   10.00		average:    9.36
Episode:  8012		score:   10.00		average:    9.36
Episode:  8013		score:   11.00		average:    9.38
Episode:  8014		score:   11.00		average:    9.38
Episode:  8015		score:    9.00		average:    9.38
Episode:  8016		score:    8.00		average:    9.36
Episode:  8017		score:   10.00		average:    9.36
Episode:  8018		score:   11.00		average:    9.39
Episode:  8019		score:    9.00		average:    9.38
Episode:  8020		score:   10.00		average:    9.39
Episode:  8021		score:   10.00		average:    9.41
Episode:  8022		score:    9.00		average:    9.40
Episode:  8023		score:    8.00		average:    9.38
Episode:  8024		score:   10.00		average:    9.38
Episode:  8025		score:    9.00		average:    9.38
Episode:  8026		score:    9.00		average:    9.38
Episode:  8027		score:    8.00		average:    9.37
Episode:  8028		score:   11.00		average:    9.38
Episode:  8029		score:   10.00		average:    9.37
Episode:  8030		score:   10.00		average:    9.38
Episode:  8031		scor

Episode:  8188		score:    8.00		average:    9.37
Episode:  8189		score:    9.00		average:    9.36
Episode:  8190		score:    9.00		average:    9.35
Episode:  8191		score:    9.00		average:    9.35
Episode:  8192		score:   10.00		average:    9.35
Episode:  8193		score:    9.00		average:    9.36
Episode:  8194		score:   10.00		average:    9.36
Episode:  8195		score:   10.00		average:    9.36
Episode:  8196		score:    8.00		average:    9.34
Episode:  8197		score:    9.00		average:    9.33
Episode:  8198		score:   11.00		average:    9.34
Episode:  8199		score:    8.00		average:    9.34
Episode:  8200		score:   10.00		average:    9.34
Episode:  8201		score:    8.00		average:    9.32
Episode:  8202		score:    8.00		average:    9.29
Episode:  8203		score:   10.00		average:    9.30
Episode:  8204		score:   10.00		average:    9.31
Episode:  8205		score:   11.00		average:    9.32
Episode:  8206		score:    9.00		average:    9.32
Episode:  8207		score:   10.00		average:    9.33
Episode:  8208		scor

Episode:  8367		score:    9.00		average:    9.36
Episode:  8368		score:    8.00		average:    9.34
Episode:  8369		score:   10.00		average:    9.35
Episode:  8370		score:    9.00		average:    9.34
Episode:  8371		score:   10.00		average:    9.36
Episode:  8372		score:   10.00		average:    9.36
Episode:  8373		score:    9.00		average:    9.35

KeyboardInterrupt: 